In [63]:
import numpy as np
import os
from numpy.random import seed
seed(1337)
#from tensorflow import set_random_seed
#set_random_seed(42)
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from tensorflow.python.keras.applications import vgg16
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras import layers, models, Model, optimizers
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [64]:
img_height, img_width = 64,64
conv_base = vgg16.VGG16(weights='imagenet', include_top=False, pooling='max', input_shape = (img_width, img_height, 3))

In [65]:
for layer in conv_base.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x00000180D74C1518> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000180D74C1E80> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000180DC849CF8> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x00000180DC899A58> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000180D7AA8748> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000180D99A0B70> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x00000180D99CAAC8> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000180D99CAB70> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000180D9A1EBE0> True
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x00000180D9A3F710> True
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x00000180D9A7AA20> True
<tensorflow.python.keras.layer

In [66]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Dense(4, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 2052      
Total params: 14,716,740
Trainable params: 14,716,740
Non-trainable params: 0
_________________________________________________________________


In [67]:
from tensorflow.keras.optimizers import Adam
learning_rate = 5e-5
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=learning_rate, clipnorm = 1.), metrics = ['acc'])

In [68]:
#Download Model
label_class = {0:'BacterialPneumonia',
               1:'COVID-19',
               2:'Normal',
               3:'ViralPneumonia'}
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing import image
def load_image(img_path, show=True):

    img = load_img(img_path, target_size=(64, 64))
    img_tensor = img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

In [69]:
try:
    import os
    import tkinter as tk
    import tkinter.ttk as ttk
    from tkinter import filedialog
    from tkinter import * 
    from PIL import ImageTk,Image  
except ImportError:
    import Tkinter as tk
    import ttk
    import tkFileDialog as filedialog

In [70]:
def c_open_image():
    global rep
    rep = filedialog.askopenfilenames(
    	parent=root,
    	initialdir='/',
    	initialfile='tmp',
    	filetypes=[
    		("PNG", "*.png"),
    		("JPEG", "*.jpg"),
    		("All files", "*")])
    print(rep)
    
    global icon
    image = Image.open(rep[0])
    width, height = image.size
    img = image.resize((round(250/height*width) , round(250)))
    photo = ImageTk.PhotoImage(img)
    icon.configure(image=photo)
    icon.image = photo # keep a reference!
    

        
        
        
def doMagic():
    test_image = rep[0] #Input Image path
    model.load_weights("sign_classifier_1.h5") #Download and provide the Model path
    new_image = load_image(test_image,False)
    pred = model.predict(new_image)
    #print(pred)
    print(label_class[np.argmax(pred[0])]) #return
    result["text"] = label_class[np.argmax(pred[0])]

In [71]:
root = tk.Tk()
root.title("COVID-19 Classifier") # Change the Title of the GUI
root.geometry('600x400') # Set the size of the Windows
title =ttk.Label(root,text ="Welcome to Corona checking Lab", font='Helvetica 18 bold')


title.pack()

xray = ttk.Label(root, text="Please upload your scan:")
xray.pack()

uploadxray = ttk.Button(root, text="Open Xray", command=c_open_image)
uploadxray.pack()
#canvas = Canvas(root, width = 300, height = 300)      
#canvas.pack()      
#img = ImageTk.PhotoImage(Image.open("Xray-icon.jpg"))      
#canvas.create_image(20, 20, anchor=NW, image=img) 
#photo = PhotoImage(file='Xray-icon.jpg')
#cv = Label(master, image=photo)
#cv.pack(side=BOTTOM)
image = Image.open("Xray-icon.jpg")
width, height = image.size
img = image.resize((round(250/height*width) , round(250)))
photo = ImageTk.PhotoImage(img)
icon = Label(image=photo)
icon.image = photo # keep a reference!
icon.pack()

submit = ttk.Button(root, text ="Check")
submit.pack()
submit.config(command = doMagic)

result = ttk.Label(root)
result.pack()


root.mainloop()


('D:/uts/Semester 6 2020/Deep learning/Assignment 2/448.jpeg',)
Normal
('D:/uts/Semester 6 2020/Deep learning/Assignment 2/Xray.jpg',)
COVID-19
